In [1]:
import random
import torch
import numpy as np
import os

def set_random_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:2"
    os.environ["PL_GLOBAL_SEED"] = str(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_random_seed(13370)

In [2]:
!head stage1_training_data.csv

,sentence1,sentence2,gold_label,sent1_readable,sent2_readable,sent1_subj_quant,sent1_subj_adj,sent1_obj_quant,sent1_obj_adj,sent1_adv,sent1_polarity,sent2_subj_quant,sent2_subj_adj,sent2_obj_quant,sent2_obj_adj,sent2_adv,sent2_polarity
8,some slow gymnast emptystring emptystring publicizes notevery emptystring straw,some emptystring gymnast emptystring jealously publicizes every opaque straw,neutral,some slow gymnast publicizes not every straw,some gymnast jealously publicizes every opaque straw,some,yes,notevery,no,no,aff,some,no,every,yes,yes,aff
12,no Spanish receiver emptystring emptystring publicizes some dry lemur,every emptystring receiver emptystring fortunately publicizes every dry lemur,contradiction,no spanish receiver publicizes some dry lemur,every receiver fortunately publicizes every dry lemur,no,yes,some,yes,no,aff,every,no,every,yes,yes,aff
13,no furry linguist emptystring deftly draws no pink lemur,every furry linguist emptystring deftly draws every emptystring lemur,

In [ ]:
# 7 sent1_subj_adj
# 9 sent1_obj_adj
# 10 sent1_adv,sent1_polarity,sent2_subj_quant,sent2_subj_adj,sent2_obj_quant,sent2_obj_adj,sent2_adv,sent2_polarity

sadj1 = set()
subj1 = set()
adv1 = set()
verb1 = set()
oadj1 = set()
obj1 = set()

sadj2 = set()
subj2 = set()
adv2 = set()
verb2 = set()
oadj2 = set()
obj2 = set()

for idx, line in enumerate(open('stage1_training_data.csv')):
    if idx:
        chunks = line.strip().split(',')
        assert len(chunks)==18, 'parse error'
        words = chunks[1].split()
        sadj1.add(words[1])
        subj1.add(words[2])
        adv1.add(words[4])
        verb1.add(words[5])
        oadj1.add(words[7])
        obj1.add(words[8])
        words = chunks[2].split()
        sadj2.add(words[1])
        subj2.add(words[2])
        adv2.add(words[4])
        verb2.add(words[5])
        oadj2.add(words[7])
        obj2.add(words[8])

set_random_seed(0xABBA+1)

sadjs = list(sadj1-set(('emptystring',)))
np.random.shuffle(sadjs)

subjs = list(subj1-set(('emptystring',)))
np.random.shuffle(subjs)

objs = list(obj1-set(('emptystring',)))
np.random.shuffle(objs)

advs = list(adv1-set(('emptystring',)))
np.random.shuffle(advs)

oadjs = list(oadj1-set(('emptystring',)))
np.random.shuffle(oadjs)

verbs = list(verb1-set(('emptystring',)))
np.random.shuffle(verbs)

TRAIN_PART = 60
VALID_PART = 1000

train_sadjs = sadjs[:TRAIN_PART]+['emptystring',]
train_subjs = subjs[:TRAIN_PART]+['emptystring',]
train_objs = objs[:TRAIN_PART]+['emptystring',]
train_advs = advs[:TRAIN_PART]+['emptystring',]
train_oadjs = oadjs[:TRAIN_PART]+['emptystring',]
train_verbs = verbs[:TRAIN_PART]+['emptystring',]

valid_sadjs = sadjs[TRAIN_PART:VALID_PART]+['emptystring',]
valid_subjs = subjs[TRAIN_PART:VALID_PART]+['emptystring',]
valid_objs =  objs[TRAIN_PART:VALID_PART]+['emptystring',]
valid_advs =  advs[TRAIN_PART:VALID_PART]+['emptystring',]
valid_oadjs = oadjs[TRAIN_PART:VALID_PART]+['emptystring',]
valid_verbs = verbs[TRAIN_PART:VALID_PART]+['emptystring',]


In [4]:
print(train_sadjs)
print(valid_sadjs)

['Ukranian', 'helpful', 'proud', 'slimy', 'Mississippian', 'German', 'Spanish', 'stupid', 'thoughtful', 'noble', 'outstanding', 'lucky', 'polite', 'Indian', 'fuzzy', 'crazy', 'devout', 'hopeful', 'Alaskan', 'English', 'Panamanian', 'Italian', 'religious', 'small', 'beautiful', 'angry', 'rude', 'Indonesian', 'insane', 'scatterbrained', 'unhinged', 'Texan', 'Korean', 'sly', 'friendly', 'idealistic', 'uncouth', 'rough', 'loving', 'Polish', 'untrustworthy', 'Austrian', 'furry', 'charismatic', 'Mongolian', 'Latvian', 'quick', 'unpredictable', 'Moroccan', 'Californian', 'kooky', 'Mexican', 'Alabaman', 'smooth', 'Afghani', 'Chinese', 'unhelpful', 'surly', 'underwhelming', 'Washingtonian', 'emptystring']
['Oregonian', 'soft', 'French', 'chummy', 'silly', 'Oklahoman', 'boisterous', 'Canadian', 'horrifying', 'coy', 'sheltered', 'taciturn', 'seductive', 'Japanese', 'vivacious', 'Israeli', 'Nebraskan', 'Siberian', 'happy', 'sneaky', 'Hawaiian', 'big', 'slow', 'sad', 'savvy', 'Lithuanian', 'Pakista

In [5]:
records = []

for idx, line in enumerate(open('stage1_training_data.csv')):
    if idx:
        chunks = line.strip().split(',')
        assert len(chunks)==18, 'parse error'
        records.append( {'left':chunks[4], 'right': chunks[5], 'label':chunks[3]})

np.random.shuffle(records)
print(len(records))

475998


In [6]:
train_records = []

for idx, line in enumerate(open('stage1_training_data.csv')):
    if idx:
        chunks = line.strip().split(',')
        assert len(chunks)==18, 'parse error'
        words = chunks[1].split()
        if  words[1] not in train_sadjs or \
            words[2] not in train_subjs or \
            words[4] not in train_advs or \
            words[5] not in train_verbs or \
            words[7] not in train_oadjs or \
            words[8] not in train_objs: continue
        words = chunks[2].split()
        if  words[1] not in train_sadjs or \
            words[2] not in train_subjs or \
            words[4] not in train_advs or \
            words[5] not in train_verbs or \
            words[7] not in train_oadjs or \
            words[8] not in train_objs: continue
        train_records.append( {'left':chunks[4], 'right': chunks[5], 'label':chunks[3]})

np.random.shuffle(train_records)
print(len(train_records))

20117


In [7]:
val_records = []

for idx, line in enumerate(open('stage1_training_data.csv')):
    if idx:
        chunks = line.strip().split(',')
        assert len(chunks)==18, 'parse error'
        words = chunks[1].split()
        if  words[1] not in valid_sadjs or \
            words[2] not in valid_subjs or \
            words[8] not in valid_objs: continue
        words = chunks[2].split()
        if  words[1] not in valid_sadjs or \
            words[2] not in valid_subjs or \
            words[4] not in valid_advs or \
            words[5] not in valid_verbs or \
            words[7] not in valid_oadjs or \
            words[8] not in valid_objs: continue
        val_records.append( {'left':chunks[4], 'right': chunks[5], 'label':chunks[3]})

np.random.shuffle(val_records)
print(len(val_records))
# val_records

3622


In [8]:
test_records = val_records[:len(val_records)//2]
val_records = val_records[-len(val_records)//2:]
print(len(test_records), len(val_records))

1811 1811


In [9]:
# VAL_SHARE = .1
# TEST_SHARE = .1

# val_records = records[:int(len(records)*VAL_SHARE)]
# test_records = records[-int(len(records)*TEST_SHARE):]
# train_records = records[int(len(records)*VAL_SHARE):-int(len(records)*TEST_SHARE)]

In [10]:
from collections import Counter

labels_counter = Counter([r["label"] for r in records])
print(labels_counter, sum(labels_counter.values()))
labels = list(labels_counter.keys())
print(labels)

Counter({'contradiction': 159754, 'neutral': 158900, 'entailment': 157344}) 475998
['entailment', 'contradiction', 'neutral']


In [11]:
import json
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer

class NLIsDataset(Dataset):
    def __init__(self, records, tokenizer, max_tokens, labels):
        self.tokenizer = tokenizer
        self.max_tokens = max_tokens
        self.records = records
        self.labels = labels
    
    def __len__(self):
        return len(self.records)
    
    def embed_record(self, record):
        inputs = self.tokenizer(
            text=record["left"],
            text_pair=record["right"],
            add_special_tokens=True,
            max_length=self.max_tokens,
            padding="max_length",
            truncation="longest_first",
            return_tensors='pt'
        )
        for key, value in inputs.items():
            value.squeeze_(0)
        return inputs
    
    def __getitem__(self, index):
        record = self.records[index]
        output = self.embed_record(record)
        label = record.get("label", None)
        if label is not None and label in self.labels:
            output["labels"] = torch.tensor(self.labels.index(label))
        return output

In [12]:
MODEL_NAME = "xlm-roberta-large"
# MODEL_NAME = 'bert-base-multilingual-uncased'
TOKENIZER_NAME = MODEL_NAME
MAX_TOKENS = 100
EPOCHS = 4*32 # 4
EVAL_STEPS = 32*4 # 32
WARMUP_STEPS = 16
LR = 0.00002
BATCH_SIZE = 32*4 # 32
GRAD_ACCUM_STEPS = 1 # 4
PATIENCE = 3

BATCH_SIZE = 32
GRAD_ACCUM_STEPS = 4


In [13]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME, do_lower_case=False)
train_data = NLIsDataset(train_records, tokenizer, MAX_TOKENS, labels)
val_data = NLIsDataset(val_records, tokenizer, MAX_TOKENS, labels)

In [14]:
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, EarlyStoppingCallback

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(labels))
model = model.to("cuda")

callbacks = [EarlyStoppingCallback(early_stopping_patience=2)]

training_args = TrainingArguments(
    output_dir="checkpoints",
    evaluation_strategy="steps",
    save_strategy="steps",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    logging_steps=EVAL_STEPS,
    save_steps=EVAL_STEPS,
    warmup_steps=WARMUP_STEPS,
    learning_rate=LR,
    num_train_epochs=EPOCHS,
    gradient_accumulation_steps=GRAD_ACCUM_STEPS,
    report_to="none",
    load_best_model_at_end=True,
    save_total_limit=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    callbacks=callbacks
)

!rm -rf checkpoints
trainer.train()

2021-11-10 20:30:01.627929: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of X

Step,Training Loss,Validation Loss
128,1.036600,0.788705
256,0.677800,0.534601
384,0.537800,0.588231
512,0.543700,0.597887


***** Running Evaluation *****
  Num examples = 1793
  Batch size = 32
Saving model checkpoint to checkpoints/checkpoint-128
Configuration saved in checkpoints/checkpoint-128/config.json
Model weights saved in checkpoints/checkpoint-128/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1793
  Batch size = 32
Saving model checkpoint to checkpoints/checkpoint-256
Configuration saved in checkpoints/checkpoint-256/config.json
Model weights saved in checkpoints/checkpoint-256/pytorch_model.bin
Deleting older checkpoint [checkpoints/checkpoint-128] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 1793
  Batch size = 32
Saving model checkpoint to checkpoints/checkpoint-384
Configuration saved in checkpoints/checkpoint-384/config.json
Model weights saved in checkpoints/checkpoint-384/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1793
  Batch size = 32
Saving model checkpoint to checkpoints/checkpoint-512
Configuration saved in che

TrainOutput(global_step=512, training_loss=0.6989619880914688, metrics={'train_runtime': 1557.4109, 'train_samples_per_second': 1631.015, 'train_steps_per_second': 12.739, 'total_flos': 2.20209475664142e+16, 'train_loss': 0.6989619880914688, 'epoch': 3.3})

In [14]:
from tqdm.notebook import tqdm

def get_batch(data, batch_size):
    start_index = 0
    while start_index < len(data):
        end_index = start_index + batch_size
        batch = data[start_index:end_index]
        yield batch
        start_index = end_index

def pipe_predict(data, pipe, batch_size=64):
    raw_preds = []
    for batch in tqdm(get_batch(data, batch_size)):
        raw_preds += pipe(batch)
    preds = np.array([int(max(labels, key=lambda x: x["score"])["label"][-1]) for labels in raw_preds])
    pp = np.array([[l["score"] for l in labels] for labels in raw_preds])
    return preds, pp

In [19]:
from transformers import pipeline
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

model.eval()
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, framework="pt", device=0, return_all_scores=True)

y_true = np.array([labels.index(r["label"]) for r in test_records], dtype=np.int32)
test_pairs = [(r["left"], r["right"]) for r in test_records]

y_pred, y_pred_prob = pipe_predict(test_pairs, pipe)

print(classification_report(y_true, y_pred, digits=3))
print(confusion_matrix(y_true, y_pred))


/media/altsoph/Volume/_py37/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


0it [00:00, ?it/s]

              precision    recall  f1-score   support

           0      0.630     0.703     0.665       667
           1      0.630     0.635     0.632       654
           2      0.939     0.782     0.853       490

    accuracy                          0.700      1811
   macro avg      0.733     0.706     0.717      1811
weighted avg      0.714     0.700     0.704      1811

[[469 198   0]
 [214 415  25]
 [ 61  46 383]]


In [15]:
### load model
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, EarlyStoppingCallback


2021-11-10 22:41:31.197354: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [16]:
records2 = []

for idx, line in enumerate(open('stage2_training_data.csv')):
    if idx:
        chunks = line.strip().split(',')
        assert len(chunks)==18, 'parse error'
        records2.append( {'left':chunks[4], 'right': chunks[5], 'label':chunks[3]})

np.random.shuffle(records2)
# records2[:5]
VAL_SHARE = .1
TEST_SHARE = .1

val_records2 = records2[:int(len(records2)*VAL_SHARE)]
test_records2 = records2[-int(len(records2)*TEST_SHARE):]
train_records2 = records2[int(len(records2)*VAL_SHARE):-int(len(records2)*TEST_SHARE)]

# tokenizer.convert_ids_to_tokens(val_data2[0]['input_ids'])[:10]

In [17]:
MAX_TOKENS = 100
EPOCHS = 4 # 4
EVAL_STEPS = 32 # 32
WARMUP_STEPS = 16
LR = 0.00002
BATCH_SIZE = 32 # 32
GRAD_ACCUM_STEPS = 4 # 4
PATIENCE = 3

In [ ]:
for run in range(40):
    print('===',run)
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, do_lower_case=False)
    tokenizer.add_tokens(['[NOT]','[FEW]','[MANY]'], special_tokens=True)
    model = AutoModelForSequenceClassification.from_pretrained('checkpoints/checkpoint-256', num_labels=len(labels))
    model = model.to("cuda")
    model.resize_token_embeddings(len(tokenizer))    
    
    np.random.shuffle(train_records2)
    train_data2 = NLIsDataset(train_records2, tokenizer, MAX_TOKENS, labels)
    val_data2 = NLIsDataset(val_records2, tokenizer, MAX_TOKENS, labels)

    print(tokenizer.convert_ids_to_tokens(val_data2[0]['input_ids'])[:10])
    
    model.train()

    callbacks = [EarlyStoppingCallback(early_stopping_patience=PATIENCE)]

    training_args = TrainingArguments(
        output_dir="checkpoints2-xlmr-mr",
        evaluation_strategy="steps",
        save_strategy="steps",
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        logging_steps=EVAL_STEPS,
        save_steps=EVAL_STEPS,
        warmup_steps=WARMUP_STEPS,
        learning_rate=LR,
        num_train_epochs=EPOCHS,
        gradient_accumulation_steps=GRAD_ACCUM_STEPS,
        report_to="none",
        load_best_model_at_end=True,
        save_total_limit=1
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_data2,
        eval_dataset=val_data2,
        callbacks=callbacks
    )

    !rm -rf checkpoints2
    trainer.train()    
    
    model.eval()
    pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, framework="pt", device=0, return_all_scores=True)

    y_true2 = np.array([labels.index(r["label"]) for r in test_records2], dtype=np.int32)
    test_pairs2 = [(r["left"], r["right"]) for r in test_records2]

    y_pred2, y_pred_prob2 = pipe_predict(test_pairs2, pipe)

    print(classification_report(y_true2, y_pred2, digits=3))
    print(confusion_matrix(y_true2, y_pred2))    
    
    not_embd = model.roberta.embeddings.word_embeddings.weight.data[[tokenizer.vocab['[NOT]']]] # .cpu().detach().numpy())
    few_embd = model.roberta.embeddings.word_embeddings.weight.data[[tokenizer.vocab['[FEW]']]]
    many_embd = model.roberta.embeddings.word_embeddings.weight.data[[tokenizer.vocab['[MANY]']]]

    np.savetxt(f'embd.not.xlmr.{run:02}.txt', not_embd.cpu().detach().numpy())
    np.savetxt(f'embd.few.xlmr.{run:02}.txt', few_embd.cpu().detach().numpy())
    np.savetxt(f'embd.many.xlmr.{run:02}.txt', many_embd.cpu().detach().numpy())

=== 0


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/xlm-roberta-large/resolve/main/config.json from cache at /home/altsoph/.cache/huggingface/transformers/4d7a1550c9ab8701667bc307a1213c040fcc06dc87a5e4994e72aecc0d7e0337.302e267433fe7c84959a639e9c7c555043daa4020c0daf311785b53de7b8685e
Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.8.2",
  "type_vocab_size"

['<s>', '▁no', '▁big', '▁prostitu', 'te', '[NOT]', '▁brush', 'es', '▁no', '▁flori']
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


***** Running training *****
  Num examples = 19200
  Num Epochs = 4
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 4
  Total optimization steps = 600


Step,Training Loss,Validation Loss
32,0.674900,0.611844
64,0.581600,0.504875
96,0.414900,0.332611
128,0.334800,0.286006
160,0.322800,0.298301
192,0.307000,0.288747
224,0.309400,0.296096


...

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/xlm-roberta-large/resolve/main/config.json from cache at /home/altsoph/.cache/huggingface/transformers/4d7a1550c9ab8701667bc307a1213c040fcc06dc87a5e4994e72aecc0d7e0337.302e267433fe7c84959a639e9c7c555043daa4020c0daf311785b53de7b8685e
Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.8.2",
  "type_vocab_size"

['<s>', '▁no', '▁big', '▁prostitu', 'te', '[NOT]', '▁brush', 'es', '▁no', '▁flori']
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


***** Running training *****
  Num examples = 19200
  Num Epochs = 4
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 4
  Total optimization steps = 600


Step,Training Loss,Validation Loss
32,0.657400,0.564354


***** Running Evaluation *****
  Num examples = 2400
  Batch size = 32
Saving model checkpoint to checkpoints2-xlmr-mr/checkpoint-32
Configuration saved in checkpoints2-xlmr-mr/checkpoint-32/config.json
Model weights saved in checkpoints2-xlmr-mr/checkpoint-32/pytorch_model.bin
Deleting older checkpoint [checkpoints2-xlmr-mr/checkpoint-576] due to args.save_total_limit
